# 🚀 REGRETTABLE ATTRITION TAHMİN PROJESİ
### End-to-End Machine Learning Pipeline ile Yüksek Performanslı Çalışan Kaybı Tahmini

**Proje Sahibi:** Arzu  
**Amaç:** F1 skoru ile yüksek performanslı çalışanların kaybını tahmin etmek  
**Versiyon:** 2.0 (Kalibrasyon + Dinamik Model Seçimi + Dual-Model Yaklaşımı)

---

In [ ]:
# --- HÜCRE 1: ORTAM KURULUMU VE VERİ YÜKLEME ---import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom google.colab import driveimport warningswarnings.filterwarnings('ignore')# Görselleştirme ayarlarıplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")# Makine Öğrenimi Kütüphanelerifrom sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFoldfrom sklearn.metrics import (classification_report, accuracy_score, f1_score,                             roc_auc_score, precision_recall_curve, confusion_matrix,                             roc_curve, average_precision_score, brier_score_loss)from sklearn.preprocessing import StandardScalerfrom sklearn.calibration import CalibratedClassifierCV, calibration_curvefrom imblearn.over_sampling import SMOTEfrom sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifierfrom sklearn.linear_model import LogisticRegressionfrom sklearn.feature_selection import SelectKBest, mutual_info_classiffrom sklearn.inspection import permutation_importanceimport lightgbm as lgbfrom numpy import nan_to_numprint("="*80)print("🚀 REGRETTABLE ATTRITION TAHMİN PROJESİ")print("="*80)# Drive'ı Bağladrive.mount('/content/drive')# Veriyi Yüklefile_path = '/content/drive/MyDrive/hr_deneme_2/HR_Analytics.csv'try:    df = pd.read_csv(file_path)    print("\n✅ Veri Başarıyla Yüklendi!")    print(f"📊 Veri Boyutu: {df.shape}")    print(f"📋 Toplam Sütun: {len(df.columns)}")    print(f"👥 Toplam Çalışan: {len(df)}")except FileNotFoundError:    print("❌ HATA: Dosya bulunamadı!")    raise SystemExitprint("\n" + "="*80)print("İLK 3 KAYIT")print("="*80)print(df.head(3))

In [ ]:
# --- HÜCRE 2: KEŞİFSEL VERİ ANALİZİ (EDA) ---print("\n" + "="*80)print(">>> KEŞİFSEL VERİ ANALİZİ (EDA)")print("="*80)# Eksik değer kontrolümissing = df.isnull().sum()if missing.sum() > 0:    print("\n⚠️ Eksik Değerler:")    print(missing[missing > 0])else:    print("\n✅ Hiç eksik değer yok")# Attrition analiziprint("\n--- Attrition Dağılımı ---")attrition_counts = df['Attrition'].value_counts()print(attrition_counts)print(f"Ayrılma Oranı: %{(attrition_counts.get('Yes', 0) / len(df) * 100):.2f}")# Performans dağılımıprint("\n--- Performans Dağılımı ---")perf_dist = df['PerformanceRating'].value_counts().sort_index()print(perf_dist)# Veri türlericategorical_cols = df.select_dtypes(include=['object']).columns.tolist()numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()print(f"\n📊 Kategorik: {len(categorical_cols)}, Sayısal: {len(numerical_cols)}")# Görselleştirmefig, axes = plt.subplots(2, 2, figsize=(15, 10))# 1. Attritionattrition_counts.plot(kind='bar', ax=axes[0,0], color=['#2ecc71', '#e74c3c'])axes[0,0].set_title('Attrition Dağılımı', fontsize=13, fontweight='bold')axes[0,0].set_ylabel('Çalışan Sayısı')axes[0,0].tick_params(rotation=0)# 2. Performanceperf_dist.plot(kind='bar', ax=axes[0,1], color='#3498db')axes[0,1].set_title('Performans Seviyesi Dağılımı', fontsize=13, fontweight='bold')axes[0,1].set_ylabel('Çalışan Sayısı')axes[0,1].tick_params(rotation=0)# 3. Yaş dağılımıdf['Age'].hist(bins=30, ax=axes[1,0], color='#9b59b6', edgecolor='black')axes[1,0].set_title('Yaş Dağılımı', fontsize=13, fontweight='bold')axes[1,0].set_xlabel('Yaş')axes[1,0].set_ylabel('Frekans')# 4. Gelir dağılımıdf['MonthlyIncome'].hist(bins=30, ax=axes[1,1], color='#e67e22', edgecolor='black')axes[1,1].set_title('Aylık Gelir Dağılımı', fontsize=13, fontweight='bold')axes[1,1].set_xlabel('Aylık Gelir')axes[1,1].set_ylabel('Frekans')plt.tight_layout()plt.show()

In [ ]:
# --- HÜCRE 3: ADIM 1 - PERFORMANS TAHMİNİ (Sızıntı Kontrolü) ---print("\n" + "="*80)print(">>> ADIM 1: Veri Sızıntısı Kontrolü - Performans Tahmini")print("="*80)# Sızıntı yapan sütunları çıkarleakage_cols = ['PercentSalaryHike', 'Attrition', 'PerformanceRating', 'Regrettable_Attrition']existing_leakage = [col for col in leakage_cols if col in df.columns]X_perf_cols = [col for col in df.columns if col not in existing_leakage]# Hedef değişken (Performance 4 ise 1, değilse 0)y_perf = df['PerformanceRating'].apply(lambda x: 1 if x == 4 else 0)print(f"Yüksek Performans (4) Oranı: %{y_perf.mean()*100:.2f}")# One-hot encodingX_perf = pd.get_dummies(df[X_perf_cols], drop_first=True)# Train/Test splitX_train_p, X_test_p, y_train_p, y_test_p = train_test_split(    X_perf, y_perf, test_size=0.2, random_state=42, stratify=y_perf)# Model eğitimirf_perf = RandomForestClassifier(    n_estimators=150, max_depth=10,    class_weight='balanced', random_state=42, n_jobs=-1)rf_perf.fit(X_train_p, y_train_p)# Değerlendirmey_pred_p = rf_perf.predict(X_test_p)y_prob_p = rf_perf.predict_proba(X_test_p)[:, 1]print(f"\n✅ Sızıntısız Model Performansı:")print(f"   Doğruluk: {accuracy_score(y_test_p, y_pred_p):.4f}")print(f"   F1 Skoru: {f1_score(y_test_p, y_pred_p):.4f}")print(f"   ROC-AUC: {roc_auc_score(y_test_p, y_prob_p):.4f}")print("\n✅ Model sızıntısız çalışıyor, gerçek performans tahmini yapabiliyor")

In [ ]:
# --- HÜCRE 4: ADIM 2 - REGRETTABLE ATTRITION TANIMI ---print("\n" + "="*80)print(">>> ADIM 2: Regrettable Attrition (Üzücü Kayıp) Tanımlama")print("="*80)# Attrition'ı sayısal yapattrition_numeric = df['Attrition'].map({'Yes': 1, 'No': 0})# REGRETTABLE ATTRITION: Ayrılan VE Performansı >=3df['Regrettable_Attrition'] = np.where(    (attrition_numeric == 1) & (df['PerformanceRating'] >= 3),    1, 0)# İstatistiklertotal_reg = df['Regrettable_Attrition'].sum()total_attrition = attrition_numeric.sum()reg_ratio = df['Regrettable_Attrition'].mean()print(f"\n📊 Regrettable Attrition İstatistikleri:")print(f"   Toplam Ayrılan: {total_attrition}")print(f"   Regrettable (Üzücü) Kayıp: {total_reg}")print(f"   Regrettable Oran: %{reg_ratio*100:.2f}")print(f"   Dengesizlik: 1:{(1-reg_ratio)/reg_ratio:.1f}")# Çapraz tabloprint("\n--- Attrition x Performance Çapraz Tablo ---")cross_tab = pd.crosstab(    df['Attrition'],    df['PerformanceRating'],    margins=True)print(cross_tab)# Görselleştirmefig, axes = plt.subplots(1, 2, figsize=(14, 5))# Regrettable Attritionreg_counts = df['Regrettable_Attrition'].value_counts()reg_counts.plot(kind='bar', ax=axes[0], color=['#95a5a6', '#e74c3c'])axes[0].set_title('Regrettable Attrition Dağılımı', fontsize=13, fontweight='bold')axes[0].set_xticklabels(['Normal (0)', 'Regrettable Kayıp (1)'], rotation=0)axes[0].set_ylabel('Çalışan Sayısı')for i, v in enumerate(reg_counts.values):    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')# Performansa göre Attritionperf_attr = df[df['Attrition']=='Yes'].groupby('PerformanceRating').size()perf_attr.plot(kind='bar', ax=axes[1], color='#e67e22')axes[1].set_title('Performans Seviyesine Göre Ayrılanlar', fontsize=13, fontweight='bold')axes[1].set_xlabel('Performance Rating')axes[1].set_ylabel('Ayrılan Sayısı')axes[1].tick_params(rotation=0)plt.tight_layout()plt.show()